## Utils


In [55]:
import random

In [56]:
LAMBDA = 1


class BikeParams:
    def __init__(self, quantum_security_level = LAMBDA, bike_variant = 1 | 2):
        if bike_variant in [1, 2]:
            if quantum_security_level == 1:
                self.n, self.r, self.w, self.t = 20326, 10163, 142, 134
            if quantum_security_level == 3:
                self.n, self.r, self.w, self.t = 39706, 19853, 206, 199
            if quantum_security_level == 5:
                self.n, self.r, self.w, self.t = 65498, 32749, 274, 264
        
        else:
            if quantum_security_level == 1:
                self.n, self.r, self.w, self.t = 22054, 11027, 134, 154
            if quantum_security_level == 3:
                self.n, self.r, self.w, self.t = 43366, 21683, 198, 226
            if quantum_security_level == 5:
                self.n, self.r, self.w, self.t = 72262, 36131, 266, 300
            
BIKE = BikeParams()


In [57]:


class Utils:

    def __init__(self) -> None:
        pass

    def odd_div_2(r = BIKE.r)->int:
        if (r // 2) % 2 == 0:
            return r//2 + 1
        return r//2




class BinaryPolynom:  # Polynom or Vector
    def __init__(self, r=BIKE.r) -> None:
        self.degree = r-1
        self.coefficients = [0 for i in range(self.degree+1)]
        pass

    def from_list(self, l):
        self.coefficients = [element for element in l]

    def get_random_polynom_with_weight(self, w = BIKE.w):
        self.coefficients[:w//2] = [1 for i in range(w//2)]
        random.shuffle(self.coefficients)

    def hamming_weight(self):
        return self.coefficients.count(1)

    def transpose(self):
        self.coefficients[1:] = self.coefficients[1:][::-1]
    
    def __str__(self) -> str:
        return "".join(str(bit) for bit in  self.coefficients)
    
    def __repr__(self) -> str:
        return str(self.coefficients)

    def __add__(a, b):
        c = BinaryPolynom(a.degree+1)
        for i in range(a.degree+1):
            c.coefficients[i] = a.coefficients[i] ^ b.coefficients[i]
        return c

    def __eq__(self, __value: object) -> bool:

        if not isinstance(__value, BinaryPolynom):
            return False
        if self.degree != __value.degree:
            return False
        
        for i in range(self.degree+1):
            if self.coefficients[i] != __value.coefficients[i]:
                return False
        return True
    
    def __sub__(a, b):
        c = BinaryPolynom(a.degree+1)
        for i in range(a.degree+1):
            c.coefficients[i] = a.coefficients[i] ^ b.coefficients[i]
        return c

    def __mul__(a, b):
        c = BinaryPolynom(a.degree+1)

        for k in range(b.degree+1):
            for j in range(k+1):
                c.coefficients[k] ^= a.coefficients[j]&b.coefficients[k-j]
            
        return c
        


In [52]:
a = BinaryPolynom(25)
a.get_random_polynom_with_weight(17)

b = BinaryPolynom(25)
b.get_random_polynom_with_weight(16)



print(a)
a.transpose()
print(a)


0000101001011000010001010
0010100010000110100101000


## Key Generation

In [58]:
h_0 = BinaryPolynom()
h_0.get_random_polynom_with_weight()

h_1 = BinaryPolynom()
h_1.get_random_polynom_with_weight()

secret_key = (h_0, h_1)

g = BinaryPolynom()
g.get_random_polynom_with_weight(Utils.odd_div_2()) # BIKE 3

public_key = (h_1*g, h_0*g) # BIKE 1 & BIKE 3

## Encryption

In [60]:
import hashlib

# Generer e0 et e1 deux polynomes tels que |e0| + |e1| = t

def brute_force_generator(t = BIKE.t):

    is_ok = False
    e0, e1 = BinaryPolynom(), BinaryPolynom()
    while not is_ok:
        e0.get_random_polynom_with_weight(random.randint(1, t))
        e1.get_random_polynom_with_weight(random.randint(1, t))
        is_ok = (e0.hamming_weight() + e1.hamming_weight() == t)
    
    return e0, e1

def brute_force_generator_2(t = BIKE.t):
    k = random.randint(1, t)
    e0, e1 = BinaryPolynom(), BinaryPolynom()
    e0.get_random_polynom_with_weight(k)
    e1.get_random_polynom_with_weight(t-k)
    
    return e0, e1

def K(e_0, e_1):
    bit_string = str(e_0) + str(e_1)
    hash = hashlib.sha3_256()
    hash.update(bit_string.encode())

    return hash.digest()[:32]

In [61]:
m = BinaryPolynom()
e_0, e_1 = brute_force_generator_2()

c_0, c_1 = m*public_key[0] + e_0, m*public_key[1] + e_1  # BIKE 1

symmetric_key = K(e_0, e_1)

In [ ]:
class OneRoundBitFlippinDecoder:
    def __init__(self) -> None:
        pass
    
    def get_residual_polynom(self, c_0, c_1, h_0, h_1):
        return c_0*h_0 + c_1*h_1

    def decode(self, s:BinaryPolynom, h_0:BinaryPolynom, h_1:BinaryPolynom):
        e = BinaryPolynom()
        s_0 = s - self.get_residual_polynom(c_0, c_1, h_0, h_1)
        T = BIKE.t//2
        for i in range(BIKE.r):
            

In [29]:
# Définir les paramètres r et t
r = 19883 # La longueur du code
t = 134 # Le poids du vecteur d'erreur

# Définir une fonction qui calcule le produit de deux polynômes modulo X^r - 1
def poly_mul(a, b):
  # Convertir les polynômes en listes de coefficients
  a = list(map(int, a))
  b = list(map(int, b))
  # Initialiser le résultat à une liste de zéros
  c = [0] * (2*r - 1)
  # Effectuer la multiplication terme à terme
  for i in range(r):
    for j in range(r):
      c[i+j] ^= a[i] & b[j] # Utiliser le xor pour l'addition dans F2
  # Réduire le résultat modulo X^r - 1
  for i in range(r-1, 0, -1):
    if c[r+i-1] == 1:
      c[r+i-1] ^= 1
      c[i-1] ^= 1
  # Retourner le résultat sous forme de chaîne de caractères
  return ''.join(map(str, c[:r]))

# Définir une fonction qui calcule le poids de Hamming d'un polynôme
def hamming_weight(a):
  # Compter le nombre de bits à 1 dans le polynôme
  return a.count('1')

# Définir une fonction qui calcule le syndrome résiduel
def residual_syndrome(s, e, h0, h1):
  # Calculer le produit eHT
  eHT = poly_mul(e, h0) + poly_mul(shift(e, 1), h1) # Utiliser le + pour le xor
  # Soustraire eHT à s
  s0 = ''.join([str(int(x) ^ int(y)) for x, y in zip(s, eHT)]) # Utiliser le xor
  # Retourner le syndrome résiduel
  return s0

# Définir une fonction qui décale un polynôme binaire de n degrés
def shift(a, n):
  # Ajouter n zéros au début de la liste
  c = '0' * n + a
  # Retirer n zéros à la fin de la liste
  c = c[:r]
  # Retourner le polynôme décalé
  return c

# Définir une fonction qui retourne un bit du polynôme
def flip(a, i):
  # Inverser le bit à la position i
  if a[i] == '0':
    a = a[:i] + '1' + a[i+1:]
  else:
    a = a[:i] + '0' + a[i+1:]
  # Retourner le polynôme modifié
  return a

# Définir une fonction qui décode un syndrome avec l'algorithme One Round Bit Flipping
def decode(s, h0, h1):
  # Initialiser le vecteur d'erreur à zéro
  e = '0' * r
  # Calculer le syndrome résiduel
  s0 = residual_syndrome(s, e, h0, h1)
  # Calculer le seuil T
  T = t // 2
  # Pour chaque bit du vecteur d'erreur
  for i in range(r):
    # Calculer le nombre de conflits
    c_i = hamming_weight(poly_mul(s0, h0 + shift(h1, i)))
    # Si le nombre de conflits est supérieur ou égal à T
    if c_i >= T:
      # Retourner le bit
      e = flip(e, i)
      # Mettre à jour le syndrome résiduel
      s0 = residual_syndrome(s, e, h0, h1)
  # Si le syndrome résiduel est nul
  if s0 == '0' * r:
    # Retourner le vecteur d'erreur
    return e
  # Sinon, retourner un symbole d'échec
  else:
    return None


b'\t\x85\xe5\ra\xe5\x1c>\xce\xfa\x93\xd7\x01\xa3R\xbe\x9e\x17\x96\x8f~\xf0\x9b\xaf\xfe5h\x16\xc8\xa0\xff\xcc'

In [ ]:

# Définir une fonction qui calcule le produit de deux polynômes modulo X^r - 1
def poly_mul(a, b):
  # Convertir les polynômes en listes de coefficients
  a = list(map(int, a))
  b = list(map(int, b))
  # Initialiser le résultat à une liste de zéros
  c = [0] * (2*r - 1)
  # Effectuer la multiplication terme à terme
  for i in range(r):
    for j in range(r):
      c[i+j] ^= a[i] & b[j] # Utiliser le xor pour l'addition dans F2
  # Réduire le résultat modulo X^r - 1
  for i in range(r-1, 0, -1):
    if c[r+i-1] == 1:
      c[r+i-1] ^= 1
      c[i-1] ^= 1
  # Retourner le résultat sous forme de chaîne de caractères
  return ''.join(map(str, c[:r]))

# Définir une fonction qui calcule le poids de Hamming d'un polynôme
def hamming_weight(a):
  # Compter le nombre de bits à 1 dans le polynôme
  return a.count('1')

# Définir une fonction qui calcule le syndrome résiduel
def residual_syndrome(s, e, h0, h1):
  # Calculer le produit eHT
  eHT = poly_mul(e, h0) + poly_mul(shift(e, 1), h1) # Utiliser le + pour le xor
  # Soustraire eHT à s
  s0 = ''.join([str(int(x) ^ int(y)) for x, y in zip(s, eHT)]) # Utiliser le xor
  # Retourner le syndrome résiduel
  return s0

# Définir une fonction qui décale un polynôme binaire de n degrés
def shift(a, n):
  # Ajouter n zéros au début de la liste
  c = '0' * n + a
  # Retirer n zéros à la fin de la liste
  c = c[:r]
  # Retourner le polynôme décalé
  return c

# Définir une fonction qui retourne un bit du polynôme
def flip(a, i):
  # Inverser le bit à la position i
  if a[i] == '0':
    a = a[:i] + '1' + a[i+1:]
  else:
    a = a[:i] + '0' + a[i+1:]
  # Retourner le polynôme modifié
  return a

# Définir une fonction qui décode un syndrome avec l'algorithme One Round Bit Flipping
def decode(s, h0, h1):
  # Initialiser le vecteur d'erreur à zéro
  e = '0' * r
  # Calculer le syndrome résiduel
  s0 = residual_syndrome(s, e, h0, h1)
  # Calculer le seuil T
  T = t // 2
  # Pour chaque bit du vecteur d'erreur
  for i in range(r):
    # Calculer le nombre de conflits
    c_i = hamming_weight(poly_mul(s0, h0 + shift(h1, i)))
    # Si le nombre de conflits est supérieur ou égal à T
    if c_i >= T:
      # Retourner le bit
      e = flip(e, i)
      # Mettre à jour le syndrome résiduel
      s0 = residual_syndrome(s, e, h0, h1)
  # Si le syndrome résiduel est nul
  if s0 == '0' * r:
    # Retourner le vecteur d'erreur
    return e
  # Sinon, retourner un symbole d'échec
  else:
    return None


In [64]:
class BinPolynom:
    def __init__(self, coeffs):
        self.coeffs = coeffs

    def __add__(self, other):
        max_len = max(len(self.coeffs), len(other.coeffs))
        coeffs_sum = [0] * max_len
        for i in range(len(self.coeffs)):
            coeffs_sum[i] ^= self.coeffs[i]
        for i in range(len(other.coeffs)):
            coeffs_sum[i] ^= other.coeffs[i]
        return BinPolynom(coeffs_sum)

    def __mul__(self, other):
        coeffs_mul = [0] * (len(self.coeffs) + len(other.coeffs) - 1)
        for i in range(len(self.coeffs)):
            for j in range(len(other.coeffs)):
                coeffs_mul[i + j] ^= self.coeffs[i] & other.coeffs[j]
        return BinPolynom(coeffs_mul)

    def __xor__(self, other):
        max_len = max(len(self.coeffs), len(other.coeffs))
        coeffs_xor = [0] * max_len
        for i in range(len(self.coeffs)):
            coeffs_xor[i] ^= self.coeffs[i]
        for i in range(len(other.coeffs)):
            coeffs_xor[i] ^= other.coeffs[i]
        return BinPolynom(coeffs_xor)

    def __str__(self):
        return ''.join(map(str, self.coeffs))

def shift(a, n):
    coeffs_shifted = [0] * n + a.coeffs[:len(a.coeffs) - n]
    return BinPolynom(coeffs_shifted)

def flip(a, i):
    coeffs_flipped = a.coeffs.copy()
    coeffs_flipped[i] ^= 1
    return BinPolynom(coeffs_flipped)

def decode(s, h0, h1):
    r = len(s.coeffs)
    e = BinPolynom([0] * r)
    s0 = residual_syndrome(s, e, h0, h1)
    T = BIKE.t // 2
    for i in range(r):
        c_i = hamming_weight(poly_mul(s0, h0 + shift(h1, i)))
        if c_i >= T:
            e = flip(e, i)
            s0 = residual_syndrome(s, e, h0, h1)
    if s0 == BinPolynom([0] * r):
        return e
    else:
        return None

def poly_mul(a, b):
    coeffs_a = a.coeffs
    coeffs_b = b.coeffs
    coeffs_mul = [0] * (len(coeffs_a) + len(coeffs_b) - 1)
    for i in range(len(coeffs_a)):
        for j in range(len(coeffs_b)):
            coeffs_mul[i + j] ^= coeffs_a[i] & coeffs_b[j]
    return BinPolynom(coeffs_mul)

def hamming_weight(a):
    return a.coeffs.count(1)

def residual_syndrome(s, e, h0, h1):
    eHT = poly_mul(e, h0) + poly_mul(shift(e, 1), h1)
    s0 = s ^ eHT
    return s0

In [65]:
s = c_0*h_0 + c_1*h_1

e = decode(BinPolynom(s.coefficients), BinPolynom(h_0.coefficients) ,BinPolynom(h_1.coefficients))

KeyboardInterrupt: 

## Decription

In [ ]:
def decode(s, h_0, h_1, u):
    

    

In [ ]:
# calcule du syndrome
s = c_0*h_0 + c_1*h_1
u = 0

# Decoder s pour retrouver (e_0_prime, e_1_prime)



symmetric_key_decapsuled = bytes()
e_0_prime, e_1_prime = BinaryPolynom(), BinaryPolynom()
if e_0_prime.hamming_weight()  + e_1_prime.hamming_weight() != BIKE.t:
    print("Decapsulation error !")
    
else:
    symmetric_key_decapsuled = K(e_0_prime, e_1_prime)
 

## BIKE Entry point

In [ ]:

# Fonction principale pour exécuter les fonctionnalités de BIKE
def main():
    # Générer les clés publiques et privées
    public_key, private_key = key_generation.generate_key_pair()

    # Message à chiffrer
    message = "Hello, World!"

    # Chiffrement du message avec la clé publique
    ciphertext = encryption.encrypt(message, public_key)

    # Déchiffrement du message chiffré avec la clé privée
    decrypted_message = decryption.decrypt(ciphertext, private_key)

    # Affichage des résultats
    print("Message clair :", message)
    print("Message chiffré :", ciphertext)
    print("Message déchiffré :", decrypted_message)

# Exécution de la fonction principale
if __name__ == "__main__":
    main()